In [1]:
import tensorflow as tf
import numpy as np

## 1. Basic of RNN-1

In [2]:
tf.reset_default_graph() # To prevent graph error of tensorflow

X_data = np.array([[[0., 1.]]])
batch_size = 1
hidden_size = 1
X = tf.placeholder(tf.float32, [None, 1, 2])

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=X, 
                                    initial_state=initial_state, dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    variables_names =[v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k,v in zip(variables_names, values):
        print(k,'\n', v)
    print('-'*30)
    outputs_val, state_val = sess.run([outputs, state], feed_dict={X:X_data})
    print('initial state:\n', sess.run(initial_state),
          '\nX_data val:\n', X_data,
          '\noutput val:\n', outputs_val,
          '\nstate val:\n', state_val)

rnn/basic_rnn_cell/kernel:0 
 [[-1.1559641 ]
 [-0.06461716]
 [ 0.81309044]]
rnn/basic_rnn_cell/bias:0 
 [0.]
------------------------------
initial state:
 [[0.]] 
X_data val:
 [[[0. 1.]]] 
output val:
 [[[-0.06452737]]] 
state val:
 [[-0.06452737]]


## 2. Basic of RNN-2

In [3]:
tf.reset_default_graph() # To prevent graph error of tensorflow

X_data = np.array([[[0., 1.],[1., 0.]]])
batch_size = 1
hidden_size = 2
X = tf.placeholder(tf.float32, [None, 2, 2])

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=X, 
                                    initial_state=initial_state, dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    variables_names =[v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k,v in zip(variables_names, values):
        print(k,'\n', v)
    print('-'*30)
    outputs_val, state_val = sess.run([outputs, state], feed_dict={X:X_data})
    print('initial state:\n', sess.run(initial_state),
          '\nX_data val:\n', X_data,
          '\noutput val:\n', outputs_val,
          '\nstate val:\n', state_val)
    
    

rnn/basic_rnn_cell/kernel:0 
 [[ 0.58285165  0.67652035]
 [ 0.55150366 -0.8950167 ]
 [ 0.60206723  0.32385635]
 [ 0.1208539  -0.03462529]]
rnn/basic_rnn_cell/bias:0 
 [0. 0.]
------------------------------
initial state:
 [[0. 0.]] 
X_data val:
 [[[0. 1.]
  [1. 0.]]] 
output val:
 [[[ 0.50164634 -0.7138628 ]
  [ 0.6632553   0.69815886]]] 
state val:
 [[0.6632553  0.69815886]]


## 3. Prediction of Basic RNN

In [4]:
tf.reset_default_graph() # To prevent graph error of tensorflow

def seqtostr(input): 
    return ''.join(ix_to_char[ch] for ch in input[:])

def sample(x, n, cell, Why, by):
    W = cell[0]
    Wxh = W[:vocab_size]
    Whh = W[vocab_size:]
    bh = cell[1]
    h = np.zeros((1, hidden_size))
    ixes = []

    for step in range(n):
        h = np.tanh(np.dot(x, Wxh) + np.dot(h, Whh) + bh)
        y = np.dot(h, Why) + by
        ix = np.argmax(y)
        x = np.zeros((1, vocab_size))
        x[0][ix] = 1
        ixes.append(ix)
    return ixes

data = 'hihello'
chars= list(set(data))
vocab_size = len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
X_data = [char_to_ix[ch] for ch in data]
X_onehot = tf.one_hot(X_data, vocab_size)

hidden_size = 10
seq_length = len(data)-1
batch_size = 1

Why = tf.Variable(tf.random_normal([batch_size, hidden_size, vocab_size], name='weight_hy'))
by = tf.Variable(tf.random_normal([batch_size, 1, vocab_size]), name='bias_y')

X = tf.placeholder(tf.float32, [None, 1, vocab_size])
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=X,
                                   initial_state=initial_state, dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    X_test = sess.run(X_onehot[0]).reshape(1, vocab_size)
    predtxt = sample(X_test, seq_length, sess.run(cell.variables), sess.run(Why), sess.run(by))
    print(data[0] + seqtostr(predtxt))

hhlllii


## 4. Prediction and Training

In [5]:
tf.reset_default_graph() # To prevent graph error of tensorflow

# For convenience
def seqtostr(input): 
    return ''.join(ix_to_char[ch] for ch in input[:])

# Kapathy's test function
def sample(x, n, cell, Why, by):
    W = cell[0]
    Wxh = W[:vocab_size]
    Whh = W[vocab_size:]
    bh = cell[1]
    h = np.zeros((1, hidden_size))
    ixes = []

    for step in range(n):
        h = np.tanh(np.dot(x, Wxh) + np.dot(h, Whh) + bh)
        y = np.dot(h, Why) + by
        # p = np.exp(y) / np.sum(np.exp(y))
        # ix = np.random.choice(list(range(vocab_size)), p=p.ravel())
        ix = np.argmax(y)
        x = np.zeros((1, vocab_size))
        x[0][ix] = 1
        ixes.append(ix)
    return ixes

# Karpathy's preprocessing
data = "hihello"

chars= list(set(data))
vocab_size = len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# Preparing training
inputs = [[char_to_ix[ch] for ch in data[:-1]]]
targets = [[char_to_ix[ch] for ch in data[1:]]]
X_train = tf.one_hot(inputs, vocab_size)
Y_train = tf.one_hot(targets, vocab_size)

# Hyperparameters
hidden_size = 10
cost_val = 100 # initial cost
learning_rate = 1e-3 #0.001

# etc.
batch_size = 1
seq_length = len(data)-1

# Placeholder
X = tf.placeholder(tf.float32, [None, seq_length, vocab_size])
Y = tf.placeholder(tf.int32, [None, seq_length, vocab_size])

# Output layer of char-rnn
Why = tf.Variable(tf.random_normal([batch_size, hidden_size, vocab_size], name='weight_hy'))
by = tf.Variable(tf.random_normal([batch_size, 1, vocab_size]), name='bias_y')

# Setting cell, loss function, optimizer
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
state = cell.zero_state(batch_size, tf.float32)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=X,
                                   initial_state=state, dtype=tf.float32)
logit = tf.matmul(outputs, Why) + by
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    for step in range(201):
        #Test
        if step % 50 == 0:
            X_test = sess.run(X_train[0][0]).reshape(1, vocab_size) # Start from the beginning
            predtxt = sample(X_test, seq_length, sess.run(cell.variables), sess.run(Why), sess.run(by))
            print('step :', step, 'cost :', cost_val,
                  '\npred :', data[0] + seqtostr(predtxt), '\n','-'*50)
            
        # Train
        cost_val, _ = sess.run([cost, train], feed_dict={X:sess.run(X_train), Y:sess.run(Y_train)})

step : 0 cost : 100 
pred : hhheeei 
 --------------------------------------------------
step : 50 cost : 0.8797013 
pred : hhheeli 
 --------------------------------------------------
step : 100 cost : 0.42367795 
pred : hihello 
 --------------------------------------------------
step : 150 cost : 0.24932496 
pred : hihello 
 --------------------------------------------------
step : 200 cost : 0.1611072 
pred : hihello 
 --------------------------------------------------
